In [23]:
import pandas as pd
from spacy.cli import download
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import spacy

In [24]:
train_data = pd.read_csv('augmented_data_with_gpt2.csv')
val_data = pd.read_csv('val_data.csv')

In [25]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented
0,https://api.github.com/repos/kubernetes/kubern...,2639668210,kubelet crash: fatal error: concurrent map writes,### What happened?\n\nWhile looking into three...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],128657.0,"['pkg/kubelet/cm/container_manager_linux.go', ...","['modified', 'modified', 'modified', 'modified']","[3, 4, 18, 75]","[3, 3, 6, 0]","[6, 7, 24, 75]",/sig node Thought to search for similar errors...,kubelet crash: fatal error: concurrent map wri...,False
1,https://api.github.com/repos/kubernetes/kubern...,2617512099,[FG:InPlacePodVerticalScaling] failed to verif...,### What happened?\n\nOne line bug description...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],126620.0,"['pkg/kubelet/status/state/checkpoint.go', 'pk...","['modified', 'modified', 'added']","[42, 23, 166]","[26, 17, 0]","[68, 40, 166]",This issue is currently awaiting triage.\n\nIf...,[fg:inplacepodverticalscaling] failed to verif...,False
2,https://api.github.com/repos/kubernetes/kubern...,2604613192,Restore build-tag flag for code-generator,### What happened?\n\nThe `build-tag` flag is ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],128259.0,['staging/src/k8s.io/code-generator/cmd/conver...,"['modified', 'modified', 'modified', 'modified...","[12, 1, 1, 12, 1, 1]","[2, 1, 1, 1, 1, 1]","[14, 2, 2, 13, 2, 2]",@p0lyn0mial FYI /sig api-machinery /triage acc...,restore build-tag flag for code-generator ### ...,False
3,https://api.github.com/repos/kubernetes/kubern...,2596132738,[Failing Tests] ci-crio-cgroupv1-node-e2e-conf...,### Which jobs are failing?\n* master-blocking...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubeadm', 'area/dependency']",128175.0,"['go.mod', 'go.sum', 'hack/unwanted-dependenci...","['modified', 'modified', 'modified', 'modified...","[1, 3, 1, 21, 9, 9, 4, 0, 10, 2, 2, 5, 8, 1]","[1, 2, 0, 98, 8, 7, 6, 62, 11, 1, 4, 8, 11, 1]","[2, 5, 1, 119, 17, 16, 10, 62, 21, 3, 6, 13, 1...",@drewhagen: The provided milestone is not vali...,[failing tests] ci-crio-cgroupv1-node-e2e-conf...,False
4,https://api.github.com/repos/kubernetes/kubern...,2591358936,Crash on kube manager's service-lb-controller ...,### What happened?\n\nIf kube manager is start...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],128182.0,['cmd/kube-controller-manager/app/controllerma...,"['modified', 'modified', 'modified']","[26, 12, 5]","[0, 1, 4]","[26, 13, 9]",This issue is currently awaiting triage.\n\nIf...,crash on kube manager's service-lb-controller ...,False


In [26]:
val_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,452179309,Some CRDs that duplicate built-in group/versio...,**What happened**:\r\n\r\nCreated a CRD:\r\n``...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/custom-resources']",82035,['staging/src/k8s.io/apiextensions-apiserver/p...,"['modified', 'modified', 'modified', 'added']","[1, 20, 9, 380]","[0, 1, 0, 0]","[1, 21, 9, 380]",@connor1989 /sub Is anyone working on this? I'...,some crds that duplicate built-in group/versio...
1,https://api.github.com/repos/kubernetes/kubern...,452175013,Some CRDs that duplicate built-in group/versio...,**What happened**:\r\nCreated a CRD with a gro...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/custom-resources', 'area/d...",81436,['staging/src/k8s.io/apiextensions-apiserver/g...,"['modified', 'modified', 'modified', 'modified...","[1, 2, 1, 14, 5, 209]","[0, 0, 0, 5, 1, 0]","[1, 2, 1, 19, 6, 209]",/cc @cheftako haven't looked into the code ye...,some crds that duplicate built-in group/versio...
2,https://api.github.com/repos/kubernetes/kubern...,451615547,Azure Disk detach backoff cancelled due to err...,**What happened**:\r\n\r\n#78298 made sure tha...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],78700,['staging/src/k8s.io/legacy-cloud-providers/az...,"['modified', 'modified']","[2, 2]","[2, 2]","[4, 4]",/assign,azure disk detach backoff cancelled due to err...
3,https://api.github.com/repos/kubernetes/kubern...,451156427,"""--kube-reserved-cgroup"" kubelet option cannot...",start kubelet with below config:\r\n```\r\n# r...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],78793,['pkg/kubelet/cm/cgroup_manager_linux_test.go'...,"['modified', 'modified']","[22, 2]","[0, 2]","[22, 4]",/area kubelet\r\n/sig node\r\n @derekwaynecarr...,"""--kube-reserved-cgroup"" kubelet option cannot..."
4,https://api.github.com/repos/kubernetes/kubern...,451147670,"Windows kubelet won't start ""Failed to create ...",**What happened**:\r\n\r\nI replaced the kubel...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/kubelet']",78704,"['pkg/kubelet/kubelet.go', 'pkg/kubelet/util/u...","['modified', 'modified', 'modified', 'modified...","[6, 2, 2, 3, 4]","[1, 2, 2, 7, 1]","[7, 4, 4, 10, 5]",cc @adelina-t \r\n/priority critical-urgent /m...,"windows kubelet won't start ""failed to create ..."


In [27]:
for i in range(5) : 
    print(train_data['all_text'][i])

kubelet crash: fatal error: concurrent map writes ### what happened?  while looking into three failing e2e tests in https://prow.k8s.io/view/gs/kubernetes-ci-logs/logs/ci-kubernetes-e2e-gci-gce-alpha-enabled-default/1854326383216431104 (an allalpha job), i found that kubelet had crashed ([logs](https://storage.googleapis.com/kubernetes-ci-logs/logs/ci-kubernetes-e2e-gci-gce-alpha-enabled-default/1854326383216431104/artifacts/bootstrap-e2e-minion-group-h288/kubelet.log)):    ```  nov 07 01:09:07.338827 bootstrap-e2e-minion-group-h288 kubelet[1937]: fatal error: concurrent map writes  nov 07 01:09:07.342609 bootstrap-e2e-minion-group-h288 kubelet[1937]: goroutine 29497 [running]:  nov 07 01:09:07.342609 bootstrap-e2e-minion-group-h288 kubelet[1937]: k8s.io/kubernetes/pkg/kubelet/cm/containermap.containermap.add(...)  nov 07 01:09:07.342609 bootstrap-e2e-minion-group-h288 kubelet[1937]:         k8s.io/kubernetes/pkg/kubelet/cm/containermap/container_map.go:36  nov 07 01:09:07.342609 boots

Check the label distribution

In [28]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in train_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Training Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Training Data):
1. area/test: 294
2. area/kubelet: 293
3. area/code-generation: 220
4. area/kubeadm: 220
5. area/dependency: 220
6. area/cloudprovider: 220
7. area/apiserver: 220
8. area/e2e-test-framework: 220
9. area/kubectl: 220
10. area/kube-proxy: 220
11. area/release-eng: 220
12. area/ipvs: 220
13. area/provider/gcp: 220
14. area/conformance: 220
15. area/provider/azure: 220
16. area/security: 220
17. area/etcd: 220
18. area/custom-resources: 220


In [29]:
# Flatten the list of labels for validation data
val_flattened_labels = []
for label_list in val_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    val_flattened_labels.extend(clean_label_list)

# Print labels distribution count for validation data
val_label_counts = Counter(val_flattened_labels)
print("Label Distribution Count (Validation Data):")

for i, (label, count) in enumerate(val_label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Validation Data):
1. area/kubelet: 79
2. area/apiserver: 45
3. area/cloudprovider: 42
4. area/test: 32
5. area/kubectl: 29
6. area/kubeadm: 18
7. area/ipvs: 13
8. area/dependency: 12
9. area/custom-resources: 9
10. area/code-generation: 6
11. area/provider/vmware: 6
12. area/kube-proxy: 6
13. area/security: 6
14. area/provider/openstack: 5
15. area/conformance: 5
16. area/provider/azure: 3
17. area/e2e-test-framework: 2
18. area/provider/gcp: 1


Stopwords Removal

In [30]:
# remove stopwords using spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))

In [31]:
# check null values in all_text column
# Display null values for all columns
print(f"Null all_text column (train): {train_data['all_text'].isnull().sum()}")
print(f"Null all_text column (val) : {val_data['all_text'].isnull().sum()}")

Null all_text column (train): 0
Null all_text column (val) : 0


Data Lemmatization

In [32]:
# lematize the text using spaCy
train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))

In [33]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Remove duplicate words

In [34]:
# remove duplicates
train_data = train_data.drop_duplicates(subset=['all_text'])
val_data = val_data.drop_duplicates(subset=['all_text'])

Remove high frequency words


In [35]:
# Merge data to get vocabulary with high-frequency words removed
merged_data = pd.concat([train_data, val_data], axis=0)

# Set the threshold for high-frequency word removal
threshold = 0.9
print(f"Processing with threshold: {threshold}")

# Create and fit CountVectorizer to get vocabulary without high-frequency words
cv = CountVectorizer(max_df=threshold)
cv.fit(merged_data['all_text'])
filtered_vocabulary = cv.vocabulary_

print(f"Vocabulary size after removing high-frequency words: {len(filtered_vocabulary)}")

Processing with threshold: 0.9
Vocabulary size after removing high-frequency words: 77191


In [36]:
# Function to filter text using the vocabulary
def filter_text_with_vocabulary(text, vocab):
    words = text.split()
    filtered_words = [word for word in words if word in vocab]
    return ' '.join(filtered_words)

# Apply filtering to train_data and val_data separately
train_data['all_text_filtered'] = train_data['all_text'].apply(
    lambda x: filter_text_with_vocabulary(x, filtered_vocabulary)
)

val_data['all_text_filtered'] = val_data['all_text'].apply(
    lambda x: filter_text_with_vocabulary(x, filtered_vocabulary)
)

# Print statistics
print("Train data statistics:")
print(f"  Average words per document: {train_data['all_text_filtered'].apply(lambda x: len(x.split())).mean():.2f}")

print("Validation data statistics:")
print(f"  Average words per document: {val_data['all_text_filtered'].apply(lambda x: len(x.split())).mean():.2f}")

Train data statistics:
  Average words per document: 166.53
Validation data statistics:
  Average words per document: 152.95


# Export the Data

In [37]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented,all_text_filtered
0,https://api.github.com/repos/kubernetes/kubern...,2639668210,kubelet crash: fatal error: concurrent map writes,### What happened?\n\nWhile looking into three...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],128657.0,"['pkg/kubelet/cm/container_manager_linux.go', ...","['modified', 'modified', 'modified', 'modified']","[3, 4, 18, 75]","[3, 3, 6, 0]","[6, 7, 24, 75]",/sig node Thought to search for similar errors...,kubelet crash : fatal error : concurrent map w...,False,kubelet crash fatal error concurrent map write...
1,https://api.github.com/repos/kubernetes/kubern...,2617512099,[FG:InPlacePodVerticalScaling] failed to verif...,### What happened?\n\nOne line bug description...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],126620.0,"['pkg/kubelet/status/state/checkpoint.go', 'pk...","['modified', 'modified', 'added']","[42, 23, 166]","[26, 17, 0]","[68, 40, 166]",This issue is currently awaiting triage.\n\nIf...,[ fg : inplacepodverticalscale ] fail verify p...,False,fg inplacepodverticalscale fail verify pod sta...
2,https://api.github.com/repos/kubernetes/kubern...,2604613192,Restore build-tag flag for code-generator,### What happened?\n\nThe `build-tag` flag is ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],128259.0,['staging/src/k8s.io/code-generator/cmd/conver...,"['modified', 'modified', 'modified', 'modified...","[12, 1, 1, 12, 1, 1]","[2, 1, 1, 1, 1, 1]","[14, 2, 2, 13, 2, 2]",@p0lyn0mial FYI /sig api-machinery /triage acc...,restore build - tag flag code - generator # # ...,False,restore build tag flag code generator happen b...
3,https://api.github.com/repos/kubernetes/kubern...,2596132738,[Failing Tests] ci-crio-cgroupv1-node-e2e-conf...,### Which jobs are failing?\n* master-blocking...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubeadm', 'area/dependency']",128175.0,"['go.mod', 'go.sum', 'hack/unwanted-dependenci...","['modified', 'modified', 'modified', 'modified...","[1, 3, 1, 21, 9, 9, 4, 0, 10, 2, 2, 5, 8, 1]","[1, 2, 0, 98, 8, 7, 6, 62, 11, 1, 4, 8, 11, 1]","[2, 5, 1, 119, 17, 16, 10, 62, 21, 3, 6, 13, 1...",@drewhagen: The provided milestone is not vali...,[ fail test ] ci-crio-cgroupv1-node-e2e-confor...,False,fail test impact multiple job job fail master ...
4,https://api.github.com/repos/kubernetes/kubern...,2591358936,Crash on kube manager's service-lb-controller ...,### What happened?\n\nIf kube manager is start...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],128182.0,['cmd/kube-controller-manager/app/controllerma...,"['modified', 'modified', 'modified']","[26, 12, 5]","[0, 1, 4]","[26, 13, 9]",This issue is currently awaiting triage.\n\nIf...,crash kube manager service - lb - controller v...,False,crash kube manager service lb controller happe...


In [38]:
# count nan values in the train_data
train_data.isna().sum()

comments_url         2718
id                      0
title                   0
body                    1
issue_url            2718
pr_url               2718
labels                  0
pr_number            2718
filename             2718
status               2718
additions            2718
deletions            2718
changes              2718
all_comments         2745
all_text                0
is_augmented            0
all_text_filtered       0
dtype: int64

In [39]:
# count nan values in the val_data
val_data.isna().sum()

comments_url          0
id                    0
title                 0
body                  0
issue_url             0
pr_url                0
labels                0
pr_number             0
filename              0
status                0
additions             0
deletions             0
changes               0
all_comments         18
all_text              0
all_text_filtered     0
dtype: int64

In [40]:
# export the train_data to csv
train_data.to_csv('preprocessed_augmented_train_data_0.9.csv', index=False)


In [41]:
val_data.to_csv('preprocessed_val_data_0.9.csv', index=False)